# Analyser data fra kompensasjonsordningen med Python og Pandas

I denne notatboken brukes data om tildelinger av støtte fra Kompensasjonsordningen for næringslivet.

Dataene ligger som både .json og .csv i dette repositoriet, men inneholder ikke de siste tallene. Vedtak skjer løpende. Oppdaterte data legges ut hver time. Dette repo-et inneholder allerede en kopi av dataene fra mars i år. Ønsker du en oppdatert versjon kan du laste ned med f.eks. curl:


In [ ]:
# JSON
%curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/json --output tilskudd.json

In [ ]:
# CSV
%curl https://data.brreg.no/kompensasjonsordning/api/tilskudd/csv --output tilskudd.csv

Steg 0: Installer Pandas

Dersom du ikke har gjort det allerede, installer Pandas (men husk den obligatoriske oppgraderingen av pip først). 

In [2]:
%pip install --upgrade pip
%pip install pandas

  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-1.2.3-cp39-cp39-manylinux1_x86_64.whl (9.7 MB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached numpy-1.20.1-cp39-cp39-manylinux2010_x86_64.whl (15.4 MB)
Note: you may need to restart the kernel to use updated packages.


Steg 1: Litt info om datasettet

Vi starter med å importere pandas, lese inn csv-filen, og få et inntrykk av hva den inneholder. Merk at det CSV-fila bruker semikolon som skilletegn.




In [3]:
import pandas as pd

df = pd.read_csv('tilskudd.csv', sep=';')
df.head()


,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
0,2021009917,913686454,AKTIV365 SANDSLI AS,4601,BERGEN,4600.0,Vestland,93.130,NaN,NaN,"125488,00",2021-03-08T00:00,november-desember2020,november-desember2019,"327907,57","766608,00","1394409,00",0.85,988782041,COLLEGIUM REVISJON AS
1,2021009916,988237612,GOLDER ASSOCIATES AS (NORWAY),3005,DRAMMEN,3000.0,Viken,71.129,NaN,NaN,"208706,41",2021-03-08T00:00,november-desember2020,november-desember2019,"554372,00","5147270,00","9239568,00",0.85,993606650,BDO AS
2,2021009915,967265403,ANITA GULBRANDSEN LØKEN,3020,NORDRE FOLLO,3000.0,Viken,49.320,NaN,NaN,"15008,28",2021-03-08T00:00,november-desember2020,november-desember2019,"34995,00","196945,00","397509,00",0.85,922829845,PARTNER REVISJON AS
3,2021009914,918718761,THAI AROMA MASSASJE AS,301,OSLO,300.0,Oslo,96.040,NaN,NaN,"7950,66",2021-03-08T00:00,november-desember2020,november-desember2019,"24905,00","81300,00","130200,00",0.85,913423585,MYHRERS RTB AS
4,2021009913,919965614,GLH LADE AS,5001,TRONDHEIM,5000.0,Trøndelag Trööndelage,47.599,NaN,NaN,"83134,37",2021-03-08T00:00,september-oktober2020,september-oktober2019,"287994,00","2328281,00","3962232,00",0.70,989271733,VEKTE AS


Vi ser at ... osv ...

Det som er mest interessant å få frem verdier på er ikke med, som tilskuddsbeløp, og ulike omsetningstall. Det skyldes at disse verdiene leses inn som tekst istedenfor tall, pga komma som desimaltegn.


In [6]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,naeringskode1,naeringskode2,naeringskode3,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9048.000000,244.000000,9.000000,9048.000000,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,58.102011,54.737045,48.966889,0.789838,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,17.751790,22.596203,29.426245,0.073522,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,0.000000,0.000000,10.710000,0.700000,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,49.320000,47.285250,15.200000,0.700000,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,56.101000,55.202000,55.202000,0.850000,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,70.220000,70.100000,78.200000,0.850000,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,96.090000,96.020000,79.909000,0.850000,9.996475e+08


In [8]:
df.sample(10)

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
8611,2021000487,988034398,GRAND EASY LIVING SOUTH AS,5402,HARSTAD,5400.0,Troms og Finnmark Romsa ja Finnmárku,55.101,NaN,NaN,"1029188,18",2021-01-19T00:00,november-desember2020,november-desember2019,"1725174,00","2743000,00","9200000,00",0.85,997398610,ENTER REVISJON HÅLOGALAND AS
4580,2021004916,912434281,ZOVENFRA AS,301,OSLO,300.0,Oslo,62.020,NaN,NaN,"29367,68",2021-02-04T00:00,november-desember2020,november-desember2019,"87804,00","486601,00","802300,00",0.85,976095057,WEPE REGNSKAP AS
4954,2021004513,914213363,PORTAL TRAVEL AS,4204,KRISTIANSAND,4200.0,Agder,79.120,NaN,NaN,"33769,08",2021-02-03T00:00,september-oktober2020,september-oktober2019,"59267,86","1059024,00","5692386,29",0.70,992093994,TET REGNSKAP OG ØKONOMI AS
1472,2021008288,945036060,MEETING MANAGEMENT AS,301,OSLO,300.0,Oslo,79.110,NaN,NaN,"34960,73",2021-02-25T00:00,september-oktober2020,september-oktober2019,"60970,00","275771,00","1524905,00",0.70,976048695,K-TEAM AS
7523,2021001699,918314008,SJØSIDEN SPORTSPUB AS,5057,ØRLAND,5000.0,Trøndelag Trööndelage,56.301,NaN,NaN,"171337,77",2021-01-22T00:00,november-desember2020,november-desember2019,"350892,51","607196,03","1426884,90",0.85,959378371,NaN
4162,2021005364,931735357,FJORD TOURS AS,4601,BERGEN,4600.0,Vestland,79.120,NaN,NaN,"379218,19",2021-02-08T00:00,november-desember2020,november-desember2019,"454613,00","347430,00","18639032,00",0.85,987009713,PRICEWATERHOUSECOOPERS AS
7903,2021001274,983540538,SOPALA AS,3802,HOLMESTRAND,3800.0,Vestfold og Telemark,56.101,NaN,NaN,"82633,57",2021-02-09T00:00,november-desember2020,november-desember2019,"304757,00","2440044,00","3883643,00",0.85,983515835,EVY HØGBERGET AS
5682,2021003706,981081803,KAFE VIGELAND AS,301,OSLO,300.0,Oslo,56.101,NaN,NaN,"84183,99",2021-01-29T00:00,september-oktober2020,september-oktober2019,"209178,00","400874,00","943079,00",0.70,987296828,RESULTAT COMPAGNIET AS
8768,2021000311,990766339,MØRK AS,301,OSLO,300.0,Oslo,96.020,NaN,NaN,"36303,79",2021-01-18T00:00,november-desember2020,november-desember2019,"121370,00","649750,00","1002549,00",0.85,971644907,REGNSKAPSHUSET AS
6325,2021003004,993528978,JORDBÆRPIKENE NERSTRANDA AS,5401,TROMSØ,5400.0,Troms og Finnmark Romsa ja Finnmárku,56.101,NaN,NaN,"80098,35",2021-01-27T00:00,november-desember2020,november-desember2019,"294198,00","1378926,00","2028745,00",0.85,977498651,AST REGNSKAP AS


In [10]:
df.columns

Index(['tilskuddsnummer', 'tilskuddsmottakerOrganisasjonsnummer',
       'tilskuddsmottakerNavn', 'kommunenummer', 'kommune', 'fylkesnummer',
       'fylke', 'naeringskode1', 'naeringskode2', 'naeringskode3',
       'tilskuddsbelop', 'tildelingsdato', 'tilskuddsperiode',
       'sammenligningsperiode',
       'sumFasteUungaaeligeKostanderITilskuddsperioden',
       'omsetningITilskuddsperiode', 'omsetningISammenligningsperiode',
       'justeringsfaktor', 'uavhengigMedsignererOrganisasjonsnummer',
       'uavhengigMedsignererNavn'],
      dtype='object')

In [18]:
df['tilskuddsbelop']


0       125488,00
1       208706,41
2        15008,28
3         7950,66
4        83134,37
          ...    
9043     20367,53
9044     56709,39
9045      9844,00
9046     11624,90
9047    156339,75
Name: tilskuddsbelop, Length: 9048, dtype: object

Må lese inn Excel-fila igjen og denne gangen presiserer vi hva som er desimalskilletegnet:

In [9]:
df = pd.read_csv('tilskudd.csv', sep=';', decimal=',')


In [10]:
df.describe()

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,kommunenummer,fylkesnummer,tilskuddsbelop,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,uavhengigMedsignererOrganisasjonsnummer
count,9.048000e+03,9.048000e+03,9048.000000,9012.000000,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03,9.048000e+03
mean,2.021005e+09,9.481143e+08,2662.386273,2651.364847,1.910668e+05,4.228766e+05,1.433093e+06,1.424610e+07,9.544368e+08
std,2.847589e+03,4.406056e+07,1772.681531,1772.489596,2.037838e+06,4.467720e+06,8.715873e+06,1.013714e+09,4.161825e+07
min,2.021000e+09,8.116188e+08,301.000000,300.000000,5.000340e+03,6.410000e+03,0.000000e+00,6.000000e+03,8.121884e+08
25%,2.021003e+09,9.168709e+08,1103.000000,1100.000000,1.718204e+04,4.087875e+04,1.117474e+05,3.516660e+05,9.206667e+08
50%,2.021005e+09,9.579039e+08,3024.000000,3000.000000,4.043306e+04,9.759950e+04,3.093570e+05,9.054335e+05,9.744623e+08
75%,2.021007e+09,9.880344e+08,4601.000000,4600.000000,1.023590e+05,2.405640e+05,9.022215e+05,2.425666e+06,9.875649e+08
max,2.021010e+09,9.996654e+08,5444.000000,5400.000000,8.188874e+07,1.918143e+08,4.146793e+08,9.640463e+10,9.996475e+08


In [11]:
df['tilskuddsbelop']

0       125488.00
1       208706.41
2        15008.28
3         7950.66
4        83134.37
          ...    
9043     20367.53
9044     56709.39
9045      9844.00
9046     11624.90
9047    156339.75
Name: tilskuddsbelop, Length: 9048, dtype: float64

In [13]:
df['tilskuddsbelop'].describe().round(2)

count        9048.00
mean       191066.80
std       2037838.13
min          5000.34
25%         17182.04
50%         40433.06
75%        102358.97
max      81888735.97
Name: tilskuddsbelop, dtype: float64

In [14]:
df[df['tilskuddsbelop']==5000.34]

,tilskuddsnummer,tilskuddsmottakerOrganisasjonsnummer,tilskuddsmottakerNavn,kommunenummer,kommune,fylkesnummer,fylke,naeringskode1,naeringskode2,naeringskode3,tilskuddsbelop,tildelingsdato,tilskuddsperiode,sammenligningsperiode,sumFasteUungaaeligeKostanderITilskuddsperioden,omsetningITilskuddsperiode,omsetningISammenligningsperiode,justeringsfaktor,uavhengigMedsignererOrganisasjonsnummer,uavhengigMedsignererNavn
6215,2021003124,982575044,SUPERLIV AS,301,OSLO,300.0,Oslo,47.919,NaN,NaN,5000.34,2021-01-27T00:00,november-desember2020,november-desember2019,22307.0,56828.0,81960.0,0.85,921575580,NUMBERS GAME AS


In [15]:
%pip install requests

  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
import requests

Hent mer info om selskapet som fikk minst støtte, gjennom API-et for åpne data fra Enhetsregisteret. Først lagrer vi URL-en til API-et.

In [17]:
APIURL = 'https://data.brreg.no/enhetsregisteret/api/enheter/'
r = requests.get(APIURL + '982575044')
r.text

KeyboardInterrupt: 